# Load List of Kings and Queens
|  
|  
v  
**!!!!! ERROR WITH Robert Brunner!!!**

In [27]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import urllib
from datetime import datetime
from pathlib import Path
import numpy as np

In [53]:
url = 'https://parlzhcdws.cmicloud.ch/parlzh2/cdws/Index/MITGLIEDER/searchdetails?q=seq>0 and %s and gremium all KR and funktion any "Mitglied Präsidium Vizepräsidium" sortBy name/sort.ascending vorname/sort.ascending&l=de-CH'

# from to
param = 'dauer_end >= "2010-11-19 00:00:00" and dauer_start <= "2022-12-31 00:00:00"'

# Get XML
r = requests.get(url % urllib.parse.quote(param))

# Load XML
tree = ET.ElementTree(ET.fromstring(r.text))
root = tree.getroot()

In [54]:
url % urllib.parse.quote(param)

'https://parlzhcdws.cmicloud.ch/parlzh2/cdws/Index/MITGLIEDER/searchdetails?q=seq>0 and dauer_end%20%3E%3D%20%222010-11-19%2000%3A00%3A00%22%20and%20dauer_start%20%3C%3D%20%222022-12-31%2000%3A00%3A00%22 and gremium all KR and funktion any "Mitglied Präsidium Vizepräsidium" sortBy name/sort.ascending vorname/sort.ascending&l=de-CH'

In [7]:
# Parse XML
ns = {
    's': 'http://www.cmiag.ch/cdws/searchDetailResponse',
    'd': 'http://www.cmiag.ch/cdws/SitzungenDetail',
    'm': 'http://www.cmiag.ch/cdws/Mitglieder'
    }

def str2dt(s):
    if s == '9999-12-31T23:59:59.000':
        s = '2100-12-31T00:00:00.000'
    return datetime.strptime(s, '%Y-%m-%dT00:00:00.000')

records = []

for mitglied in root.findall('.//m:Mitglied', ns):
    person = mitglied.find('m:Person', ns)

    end_person = mitglied.find('m:Dauer/m:End', ns).text
    if end_person == '9999-12-31T23:59:59.000':
            # Set 2100 als enddate
            end_person = '2100-12-31T00:00:00.000'

    start_person = datetime.strptime(mitglied.find('m:Dauer/m:Start', ns).text, '%Y-%m-%dT00:00:00.000')


    # Loop for each Party. If someone switch party, add person twice
    for party in person.findall('m:Kontakt/m:Parteizugehoerigkeiten/m:Parteizugehoerigkeit', ns):
        end = party.find('m:Dauer/m:End', ns).text
        if end == '9999-12-31T23:59:59.000':
            # Set end_person as end_date
            end = end_person
        end = datetime.strptime(end, '%Y-%m-%dT00:00:00.000')

        start = datetime.strptime(party.find('m:Dauer/m:Start', ns).text, '%Y-%m-%dT00:00:00.000')
        if start < start_person:
            start = start_person

        record = {
            'name': person.find('m:Kontakt/m:Name', ns).text,
            'vorname': person.find('m:Kontakt/m:Vorname', ns).text,
            'geschlecht': person.find('m:Kontakt/m:Geschlecht', ns).text,
            'wahlkreis': person.find('m:Kontakt/m:Wahlkreis', ns).text,
            'partei': party.find('m:Name', ns).text,
            'start': start,
            'end': end,
        }

        records.append(record)

    #print(datetime.strptime(mitglied.find('m:Dauer/m:End', ns).text, '%Y-%m-%dT00:00:00.000'))
    #break

df = pd.DataFrame(records)

# Save
df.to_csv(Path('../export/mitglieder.csv'), index=False)

## Download Präsidium
**!!!!! CORRUPT CSV!!!!!!!**

In [47]:
# ANPASSEN! https://raw.githubusercontent.com/openZH/krdb-lod/master/data/funktionen.csv
df_funktion = pd.read_csv('/Users/simon/Downloads/funktionen.csv', encoding='utf-8')
#df_person = pd.read_csv('https://raw.githubusercontent.com/openZH/krdb-lod/master/data/personen.csv')
df_person = pd.read_csv('/Users/simon/Downloads/personen.csv')

# Filter
df_funktion = df_funktion[df_funktion.FUNKTIONSBEZEICHNUNG.isin(['1. Vizepräsidium', 'Präsidium', '2. Vizepräsidium'])]

In [52]:
df_merged = df_funktion.merge(df_person, on = 'ID_PERSON_NEW')

df_merged['start'] = df_merged.apply(lambda row: "%s-%2d-%2d" % (row['DATUM_VON_JAHR'], row['DATUM_VON_MONAT'], row['DATUM_VON_TAG']), axis=1)
df_merged['end'] = df_merged.apply(lambda row: "%s-%2d-%2d" % (row['DATUM_BIS_JAHR'], row['DATUM_BIS_MONAT'], row['DATUM_BIS_TAG']), axis=1)

df_merged = df_merged[['VORNAME', 'NACHNAME', 'GESCHLECHT', 'FUNKTIONSBEZEICHNUNG', 'start', 'end']]
df_merged.rename(columns={'VORNAME': 'vorname', 'NACHNAME': 'nachname', 'GESCHLECHT': 'geschlecht', 'FUNKTIONSBEZEICHNUNG': 'funktion'}, inplace=True)

df_merged.to_csv(Path('../export/funktion.csv'), index=False)

,VORNAME,NACHNAME,GESCHLECHT,FUNKTIONSBEZEICHNUNG
0,Albert L.,Petermann,m,NaN
1,Jacques,Ruedin,m,NaN
2,Ernst,Nievergelt,m,NaN
3,Hansheinrich,Reimann,m,NaN
4,Otto,Fischer,m,NaN


## Last ***** attempt: Via XLSX file from Staatsarchiv
Download at the bottom: https://www.zh.ch/de/politik-staat/wahlen-abstimmungen/kantons-regierungsratswahlen/mitglieder-kantonsrats-ab-1803.html#-373901206

In [66]:
df_einsitz = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Einsitze')
df_person = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Personen')
df_partei = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Parteien')

In [72]:
df = df_partei.merge(df_person, on = 'ID_PERSON_NEW')

records = []

# Select 2000
#df = df[df.DATUM_EINTRITT_JAHR >= 2000]

for i, row in df.iterrows():
    #start = None
    #end = None

    try:
        if not np.isnan(row['DATUM_VON_JAHR']) and not(np.isnan(row['DATUM_VON_MONAT'])):
            start = datetime(int(row['DATUM_VON_JAHR']), int(row['DATUM_VON_MONAT']), int(row['DATUM_VON_TAG']))



        if (not np.isnan(row['DATUM_BIS_JAHR'])) and (not np.isnan(row['DATUM_BIS_MONAT'])):
            end = datetime(int(row['DATUM_BIS_JAHR']), int(row['DATUM_BIS_MONAT']), int(row['DATUM_BIS_TAG']))
    except Exception as e:
        print(e)
        print(row)
        row.to_clipboard()
        raise
    #start = datetime.strptime(party.find('m:Dauer/m:Start', ns).text, '%Y-%m-%dT00:00:00.000')

    # Find Party. Take oldest one.
    #df_p = df_partei[df_partei.ID_PERSON_NEW == row['ID_PERSON_NEW']].sort_values('DATUM_VON_JAHR')
    if len(df_p) == 0:
        raise('No Party', row)

    record = {
        'name': row['NACHNAME'],
        'vorname': row['VORNAME'],
        'geschlecht': row['GESCHLECHT'],
        'jahrgang': row['DATUM_GEBURT_JAHR'],
        'partei': row['PARTEIBEZEICHNUNG'],
        'start': datetime.strftime(start, '%Y-%m-%dT00:00:00.000'),
        'end': datetime.strftime(end, '%Y-%m-%dT00:00:00.000'),
    }
    records.append(record)

df_res = pd.DataFrame(records)
df_res.to_csv(Path('../export/mitglieder.csv'), index=False)

print("finito")

finito


## DEPRICATED

In [46]:
df = df_einsitz.merge(df_person, on = 'ID_PERSON_NEW')

# Select 2000
df = df[df.DATUM_EINTRITT_JAHR >= 2000]

df['DATUM_EINTRITT_JAHR'] = df['DATUM_EINTRITT_JAHR'].astype(int)
df['DATUM_EINTRITT_MONAT'] = df['DATUM_EINTRITT_MONAT'].astype(int)
df['DATUM_EINTRITT_TAG'] = df['DATUM_EINTRITT_TAG'].astype(int)

records = []

for i, row in df.iterrows():

    start = datetime(row['DATUM_EINTRITT_JAHR'], row['DATUM_EINTRITT_MONAT'], row['DATUM_EINTRITT_TAG'])

    if not np.isnan(row['DATUM_AUSTRITT_JAHR']):
        end = datetime(int(row['DATUM_AUSTRITT_JAHR']), int(row['DATUM_AUSTRITT_MONAT']), int(row['DATUM_AUSTRITT_TAG']))

    #start = datetime.strptime(party.find('m:Dauer/m:Start', ns).text, '%Y-%m-%dT00:00:00.000')

    # Find Party. Take oldest one.
    df_p = df_partei[df_partei.ID_PERSON_NEW == row['ID_PERSON_NEW']].sort_values('DATUM_VON_JAHR')
    if len(df_p) == 0:
        raise('No Party', row)

    record = {
        'name': row['NACHNAME'],
        'vorname': row['VORNAME'],
        'geschlecht': row['GESCHLECHT'],
        'jahrgang': int(row['DATUM_GEBURT_JAHR']),
        'partei': df_p.iloc[0]['PARTEIBEZEICHNUNG'],
        'start': datetime.strftime(start, '%Y-%m-%dT00:00:00.000'),
        'end': datetime.strftime(end, '%Y-%m-%dT00:00:00.000'),
    }
    records.append(record)

df_res = pd.DataFrame(records)
df_res.to_csv(Path('../export/mitglieder.csv'), index=False)